# AI To Play Mine Sweeper

## The Game Object

I developed the pythonic minesweeper. It is unit tested fairly throughly; up until visualizations and tracking features of the project present themselves. Note: The 9's are bombs; because it is impossible for a cell to be sourounded by more Bombs then there are cells surronding it.

In [1]:
import game
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from ipywidgets import IntProgress
from IPython.display import display
import random

In [2]:
from importlib import reload

reload(game)

<module 'game' from '/home/lukenelson/dev/MineSweeperAI/game.py'>

In [3]:
mygame = game.minesweeper(14, 8, 25)


for row in mygame.board:
    for c in row:
        print(c, "\t", end="")
    print("\n")
#mygame.clickCell((0,0))

0.0 	0.0 	0.0 	0.0 	1.0 	1.0 	1.0 	0.0 	1.0 	2.0 	9.0 	1.0 	0.0 	0.0 	

0.0 	1.0 	1.0 	1.0 	1.0 	9.0 	3.0 	2.0 	2.0 	9.0 	2.0 	2.0 	1.0 	1.0 	

0.0 	1.0 	9.0 	1.0 	2.0 	4.0 	9.0 	9.0 	2.0 	1.0 	1.0 	1.0 	9.0 	1.0 	

0.0 	2.0 	2.0 	3.0 	3.0 	9.0 	9.0 	4.0 	1.0 	0.0 	1.0 	3.0 	3.0 	2.0 	

0.0 	1.0 	9.0 	2.0 	9.0 	9.0 	9.0 	2.0 	0.0 	0.0 	2.0 	9.0 	9.0 	1.0 	

1.0 	2.0 	2.0 	4.0 	6.0 	9.0 	5.0 	2.0 	1.0 	0.0 	3.0 	9.0 	4.0 	1.0 	

9.0 	2.0 	1.0 	9.0 	9.0 	9.0 	3.0 	9.0 	1.0 	0.0 	2.0 	9.0 	3.0 	1.0 	

9.0 	2.0 	1.0 	2.0 	3.0 	2.0 	2.0 	1.0 	1.0 	0.0 	1.0 	2.0 	9.0 	1.0 	



## Visualizing the Game

The following is a rough gui for the game. Feel free to mess around; when you loose all of the cells will disable and you will be alerted. When you win you will be alerted. Note: If you want to start a new game you will have to run the cell above because you need to make a new object to start over

In [4]:
mygame.playOnNoteBook()

## Neruel Network to Play Mine Sweeper (Classification Approach) 

We will attempt to play minesweeper by generating lots of games (which will be used as our data). 

Things that we'll need to do:

Pad the game board with some element.
This way we distiguish the edges of the game board (all of our data needs to have the same number of features) 

Represent "unknown squares" in our classification we will have some data that are "completed games" (the whole board is visible) and others where the game is 80% finished; 60%; 40% and 20%. (Our "Solve Percent" utility will help us with this)

In [5]:
mygame = game.minesweeper(14, 8, 25)
mygame.solvePercent(.2)
mygame.playOnNoteBook()

In [6]:
theData = []
N = 2 #N represents the number of surronding squares to get


for p in [.8, .7, .6, .5, .4, .3, .2, .1]:
    for i in range(1500):
        bomb_ratio = random.uniform(.1, .25)
        width = random.randint(8,16)
        height = random.randint(8,16)
        mygame = game.minesweeper(width, height, height*width*bomb_ratio)
        mygame.solvePercent(p)
        dataToAdd = np.pad(mygame.visible, (N), 'constant', constant_values=(-1))
        dataToAdd = np.where(dataToAdd=='*', 0, dataToAdd)
        theData.append(dataToAdd)
        
for i in range(1500):
    mygame = game.minesweeper(16, 30, 99)
    dataToAdd = np.pad(mygame.board, (N), 'constant', constant_values=(-1))
    theData.append(dataToAdd.astype('<U3'))


In [7]:
theRealData = []


for data in theData:
    innerData = data[N:-N, N:-N]
    for index, x in np.ndenumerate(innerData):
        toAppend = []
        toAppend.append(x)
        for i in range((2*N+1)**2):
            if i != (((2*N+1)**2)//2):
                a = i//(2*N+1)
                b = i%(2*N+1)
                toAppend.append(data[index[0] + a, index[1] + b])
        theRealData.append(toAppend)

(Comment this line out if you don't want to eliminate the unknown value prediction)

In [8]:
theRealData = np.array(theRealData)
theRealData = theRealData[theRealData[:, 0] != '?']

This next line will make it so that it's classifying not bomb (which is true) and bomb (false) -- so if a cell is a bomb it will give it a false value. Think of it as an "okay to click" classification.

In [9]:
theRealData[:, 0] = ['1.0' if r else '0.0' for r in (theRealData[:, 0] != '9.0')]
theRealData

array([['1.0', '-1', '-1', ..., '?', '?', '-1'],
       ['1.0', '-1', '-1', ..., '?', '?', '?'],
       ['1.0', '-1', '-1', ..., '?', '?', '?'],
       ...,
       ['1.0', '1.0', '1.0', ..., '-1.', '-1.', '-1.'],
       ['0.0', '1.0', '2.0', ..., '-1.', '-1.', '-1.'],
       ['1.0', '2.0', '1.0', ..., '-1.', '-1.', '-1.']], dtype='<U3')

In [10]:
theRealData = np.where(theRealData=='?', 10, theRealData)
X, y = theRealData[:, 1:].astype(float).astype(str), theRealData[:, 0].astype(float).astype(str)

enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(X)

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)

In [11]:
#print([cat.shape for cat in enc.categories_])
print(enc.categories_)
X = enc.transform(X).toarray()

[array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.

In [12]:
X.shape

(1748393, 264)

In [13]:
y.shape

(1748393,)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [15]:
"""unique, counts = np.unique(y_train, return_counts=True)
#print(unique, counts)
oneindes = np.where(y_train=='1.0')[0]
np.random.shuffle(oneindes)
newTrainIndies = np.append(oneindes[:counts[0]], np.where(y_train=='0.0')[0])
#np.unique(y_train[newTrainIndies], return_counts=True)
y_train = y_train[newTrainIndies]
X_train = X_train[newTrainIndies]"""

"unique, counts = np.unique(y_train, return_counts=True)\n#print(unique, counts)\noneindes = np.where(y_train=='1.0')[0]\nnp.random.shuffle(oneindes)\nnewTrainIndies = np.append(oneindes[:counts[0]], np.where(y_train=='0.0')[0])\n#np.unique(y_train[newTrainIndies], return_counts=True)\ny_train = y_train[newTrainIndies]\nX_train = X_train[newTrainIndies]"

The Hidden Layer "24" is inspired by the idea that each surrounding node has been one-hot-encoded into 12 features

In [16]:
clf = MLPClassifier(solver='adam', activation="tanh", alpha=1e-5, hidden_layer_sizes=(100), verbose=True, tol=.01)

In [17]:
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Iteration 1, loss = 0.08933053
Iteration 2, loss = 0.05101188
Iteration 3, loss = 0.04503993
Iteration 4, loss = 0.04231931
Iteration 5, loss = 0.04055020
Iteration 6, loss = 0.03922326
Iteration 7, loss = 0.03818998
Iteration 8, loss = 0.03736243
Iteration 9, loss = 0.03657510
Iteration 10, loss = 0.03593638
Iteration 11, loss = 0.03532849
Iteration 12, loss = 0.03479740
Iteration 13, loss = 0.03431415
Training loss did not improve more than tol=0.010000 for 10 consecutive epochs. Stopping.
0.9830563114200045
[[ 72639   7158]
 [  2618 494555]]


In [18]:
clf.predict_proba(X_test[0].reshape(1, -1))[0][1]

0.9999203420246839

# The AI that Plays Mine Sweeper

In [19]:
wins = 0
loses = 0
gamesToPlay = 100
bombGuessTol = .85

class getBoundry:
    def __init__(self):
        self.visited = []
        self.toReturn = []
        self.perim = []
    def getBoundry(self, gameObj, index):
        if gameObj.visible[index[0]][index[1]] !="?" and index not in self.visited:
            self.visited.append(index)
            self.toReturn.append(index)
            [self.getBoundry(gameObj, i) for i in gameObj.getSurroundingSquares(index)]
        elif index not in self.visited and (index[0] <= gameObj.width and index[1] <= gameObj.height):
            self.visited.append(index)
            self.toReturn.append(index)
            self.perim.append(index)
        return self.toReturn


f = IntProgress(min=0, max=gamesToPlay) # instantiate the bar
display(f) # display the bar

for _ in range(gamesToPlay):
    f.value += 1
    mygame = game.minesweeper(9, 9, 10, record=True)
    #mygame.playOnNoteBook()

    #Our algorithm selects (0,0) first every time any way -- input is all the same on all unknown...
    #To test if there is a better place than this think about classifying better starting points... Differn't shapes...
    mygame.clickCell((1,1))#gameboard[1][1].click()
    innerData = data[N:-N, N:-N]

    while ("?" in " ".join([" ".join(map(str, row)) for row in mygame.visible])):
        curbest = 0
        bestindex = (0, 0)
        data = np.array(mygame.visible).astype('<U3')
        data = np.pad(data, (N), 'constant', constant_values=(-1))
        data = np.where(data=='*', 0, data)
        boundObject = getBoundry()
        boundObject.getBoundry(mygame, (0,0))
        print(len(boundObject.getBoundry(mygame, (0,0))))
        boundry = sorted(boundObject.perim)
        for index, x in np.ndenumerate(innerData):
            if x == "?":
                toAppend = []
                for i in range((2*N+1)**2):
                    if i != (((2*N+1)**2)//2):
                        a = i//(2*N+1)
                        b = i%(2*N+1)
                        toAppend.append(data[index[0] + a, index[1] + b])
                toAppend = np.array(toAppend)
                toAppend = np.where(toAppend=='?', 10, toAppend)
                toAppend = toAppend.astype(float).astype(str)
                toAppend = toAppend.reshape(1,-1)
                toAppend = enc.transform(toAppend).toarray()
                if clf.predict_proba(toAppend)[0][1] > curbest:
                    curbest = clf.predict_proba(toAppend)[0][1]
                    bestindex = index
                if clf.predict_proba(toAppend)[0][0] > bombGuessTol:
                    #print(index)
                    mygame.visible[index[0]][index[1]] = 9.0
                    #mygame.gameboard[index[0]][index[1]].description = "9.0"
        #print(curbest)
        #print(bestindex)
        mygame.clickCell(bestindex)
        #mygame.gameboard[bestindex[0]][bestindex[1]].click()
        #mygame.clickCell(bestindx)

        #print(c, bestindex)
    if mygame.visible == "You have lost":
        loses += 1
    else:
        wins += 1

print("The AI won {} times and lose {} times".format(wins, loses))        
    
#print(mygame.visible)

IntProgress(value=0)

The AI won 63 times and lose 37 times


In [20]:
mygame = game.minesweeper(30, 16, 99, record=True)
mygame.playOnNoteBook()

#Our algorithm selects (0,0) first every time any way -- input is all the same on all unknown...
#To test if there is a better place than this think about classifying better starting points... Differn't shapes...
#mygame.clickCell((1,1))
mygame.gameboard[1][1].click()

while ("?" in " ".join([" ".join(map(str, row)) for row in mygame.visible])):
    curbest = 0
    bestindex = (0, 0)
    data = np.array(mygame.visible).astype('<U3')
    data = np.pad(data, (N), 'constant', constant_values=(-1))
    data = np.where(data=='*', 0, data)
    max_right = max(np.where(np.array(mygame.visible) != '?')[0])+1 
    max_down = max(np.where(np.array(mygame.visible) != '?')[1])+1
    innerData = (data[N:-N, N:-N])[0:max_right+1, 0:max_down+1] 
    for index, x in np.ndenumerate(innerData):
        if x == "?":
            toAppend = []
            for i in range((2*N+1)**2):
                if i != (((2*N+1)**2)//2):
                    a = i//(2*N+1)
                    b = i%(2*N+1)
                    toAppend.append(data[index[0] + a, index[1] + b])
            toAppend = np.array(toAppend)
            toAppend = np.where(toAppend=='?', 10, toAppend)
            toAppend = toAppend.astype(float).astype(str)
            toAppend = toAppend.reshape(1,-1)
            toAppend = enc.transform(toAppend).toarray()
            if clf.predict_proba(toAppend)[0][1] > curbest:
                curbest = clf.predict_proba(toAppend)[0][1]
                bestindex = index
            if clf.predict_proba(toAppend)[0][0] > bombGuessTol:
                #print(index)
                #mygame.visible[index[0]][index[1]] = 9.0
                mygame.gameboard[index[0]][index[1]].description = "9.0"
    #print(curbest)
    #print(bestindex)
    #mygame.clickCell(bestindex)
    mygame.gameboard[bestindex[0]][bestindex[1]].click()
    #mygame.clickCell(bestindx)

'You have lost'

In [21]:
mygame.visible


'You have lost'

# Idea's to Improve:
### Mess with Hyper Params
### Mess with the solve function (make exact percentages etc)
### Mess with what we are classifying (We just want to know if something is a bomb or not...
### More Data surronding each point
### All unique Data

## Oberservations
Just because a classifier is good doesn't make it good for the game

# Messing with Hyper Params

In [22]:
#clf2 = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

#clf2.fit(X_train, y_train)

#y_pred = clf2.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [23]:
# This one is inspired by the one hot incoding (Each feature was split into 11 parts)
#clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(11), random_state=1)

#clf3.fit(X_train, y_train)

#y_pred = clf3.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [24]:
#import pickle

#filename = 'clf1.sav'
#pickle.dump(clf, open(filename, 'wb'))

#filename = 'clf2.sav'
#pickle.dump(clf2, open(filename, 'wb'))

#filename = 'clf3.sav'
#pickle.dump(clf3, open(filename, 'wb'))

In [25]:
#clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(8, 3), random_state=1)

#clf3.fit(X_train, y_train)

#y_pred = clf3.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [26]:
#filename = 'clf3.sav'
#pickle.dump(clf3, open(filename, 'wb'))

# Messing with what we are classifying

In [27]:
np.unique(y.shape)

array([1748393])